In [ ]:
# default_exp scanstat

In [ ]:
#export
import numpy as np
import pandas as pd
from scipy import special
from scipy.stats import beta

# scanstat

> The functions that help detect clusters in point processes

In [ ]:
#export
def correct_intervals(data):
    '''
    returns elements of all possible correct intervals, assumes `data` is sorted, small to large
    '''
    a = data
    n = len(data)
    a = list(range(0, n))
    b = list(range(0, n))
    c = []
    d = []
    for i in range(0, len(a)):
        b.pop(0)
        a.pop()
        c.extend(a)
        d.extend(b)
    X_i = c
    X_j = d
    
    return c, d


def normalize(a):
    '''
    Normalize to [0, 1]
    '''
    a = sorted([float(ai) for ai in a])
    normalized_a = []
    y = min(a)
    z = max(a)
    normalized_a = [(ai - y) / (z - y) for ai in a] 
    
    return normalized_a


def ibf(x, a, b): 
    '''
    Incomplete beta function, vectorized
    '''
    return beta.cdf(x = x, a = a,  b = b) * special.beta(a, b)


def transform(cluster_location, data, verbose=False):
    '''
    Trainsform values around significant cluster
    '''
    i = cluster_location[0]
    j = cluster_location[1]
    n = len(data)
    t = 1 - data[j] + data[i]
    
    new_data = []
    
    for k in range(n):
        if ((0 <= k) and (k <= i)): 
            new_data.append(data[k] / t)
        elif (((i + 1) <= k) and (k <= ((n - 1) - j + i))): 
            # n-1 important here because len(data) is greater than the numeral of the last element in data, i.e., if len(data) = 10, data[10] will fail
            new_data.append((data[k + j - i] - data[j] + data[i]) / t)
            
        if verbose == True: 
            print (f'repositioning data: [ {k} : , {data[k]} ] \n')
            
    return new_data


def get_cluster(data, type="min", stat_dist=False): # cluster detection
    '''
    Find initial cluster
    '''
    if len(data) < 3:
        raise Exception('Need more than two data points')
        
    if stat_dist == True: 
        data = list(np.random.uniform(0, 1, len(data))) #
        
    data = list(set(data)) # remove duplicated values
    n = len(data)
    data = sorted(data)
    data = normalize(data)

    #get all correct intervals
    i, j = correct_intervals(data)
    
    #compute the statistic
    I_H_F = float(1) / ibf((np.array(data)[j] - np.array(data)[i]), np.array(j) - np.array(i), (n) + 1 - np.array(j) + np.array(i))
    I_H_F = list(I_H_F)
    I_H_F_max_loc = I_H_F.index(max(I_H_F))
    I_H_F_min_loc = I_H_F.index(min(I_H_F))
  
    full_data_max = {
        'X_i': [data[i[I_H_F_max_loc]]],
        'X_j': [data[j[I_H_F_max_loc]]],
        'i': [i[I_H_F_max_loc]],
        'j': [j[I_H_F_max_loc]],
        'I_H_F': [I_H_F[I_H_F_max_loc]]
    }

    full_data_min = {
        'X_i': [data[i[I_H_F_min_loc]]],
        'X_j': [data[j[I_H_F_min_loc]]],
        'i': [i[I_H_F_min_loc]],
        'j': [j[I_H_F_min_loc]],
        'I_H_F': [I_H_F[I_H_F_min_loc]]
    }
  
    if type == 'max': return full_data_max 
    if type == 'min': return full_data_min


def get_ps(data, clust_table, permutations, type='max'):
    '''
    Compute p-value
    '''
    if len(data) < 3 : return clust_table
    else:
        L = []
        for i in range(permutations):
            table = get_cluster(data, stat_dist=True)
            L.append(table['I_H_F'][-1])

        L = sorted(L)
        if type == 'max': 
            A_H_F_p = 1. - sum(1. * ((np.array(L) < clust_table["I_H_F"][-1])) / len(L))
        if type == 'min': 
            A_H_F_p = sum(1. * ((np.array(L) < clust_table["I_H_F"][-1])) / len(L))

        clust_table['ps'][-1] = A_H_F_p

        return clust_table    

def get_clusters(data, type='max', permutations=1000, p=0.05):
    clust_table = get_cluster(data, type=type)
    data = list(set(data))
    data = sorted(data)
    data = normalize(data)
    clust_table['ps'] = [float(0.0)]
    clust_table = get_ps(data, clust_table, permutations, type=type)
    
    while (clust_table['ps'][-1] < p):
        i = clust_table['i'][-1]
        j = clust_table['j'][-1]
        
        data = transform([i, j], data)
        
        new_clust_table = get_cluster(data)
        new_clust_table['ps'] = [float(0.0)]
        new_clust_table_with_ps = get_ps(data, new_clust_table, permutations, type=type)
        
        for ii in new_clust_table_with_ps.keys():
            clust_table[ii].extend(new_clust_table_with_ps[ii])
            
    return pd.DataFrame(clust_table).pipe(lambda x: x[x.loc[:, 'ps'] <= p])

# Hypothesis free scan statistic

Python implementation of the hypothesis-free multiple scan statistic with variable window for detecting clusters in point processes (L. Cucala. A hypothesis-free multiple scan statistic with variable window. Biom. J., 50 (2008), pp. 299-310). The algorithm uses a Monte Carlo procedure to compute p-values.

See the below example using data from Knox, G. (1959). Secular pattern of congenital oesophageal atresia. British Journal of Preventive Social Medecine 13, 222–226.

In [ ]:
knox_1959 = [170, 316, 445, 468, 938, 1034, 1128, 1233, 1248, 1249,
        1252, 1259, 1267, 1305, 1385, 1388, 1390, 1446, 1454,
        1458, 1461, 1491, 1583, 1699, 1702, 1787, 1924, 1974,
        2049, 2051, 2067, 2075, 2108, 2151, 2174, 2191]

clusters = get_clusters(knox_1959)

In [ ]:
clusters

,X_i,X_j,i,j,I_H_F,ps
0,0.525977,0.653637,7,21,7.319199e+14,-6.661338e-16


In [ ]:
print(knox_1959[7], knox_1959[21])

1233 1491
